In [2]:
import pandas as pd
import os
import math

In [2]:
pip install fsspec

Note: you may need to restart the kernel to use updated packages.


In [3]:
my_current_directory = os.getcwd()
my_current_directory

'/Users/liyuhan/IPadress-polarization'

In [234]:
# segmenting the file into several files according to its meta-post
mydf = pd.read_excel('./huxijin-replies.xlsx',sheet_name = 0)

In [235]:
DataList = list(mydf.groupby(['<fullpath>']))

In [236]:
for data in DataList:
    data[1].to_csv('/Users/liyuhan/IPadress-polarization/classify/huxijin/all_file/' + str(data[0])+ '.csv', index=False)

## 2.0 Sentiment Analysis
### 2.1 snownlp

In [36]:
pip install snownlp

     |████████████████████████████████| 37.6 MB 6.0 MB/s eta 0:00:012
  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760963 sha256=895c521518a92a3b793724dc61c25945a03adcc2eae58a55d5763e2f4d63e9d8
  Stored in directory: /Users/liyuhan/Library/Caches/pip/wheels/4a/7a/fe/a5747434679b22a95c93bcf9fa49a988f5d9be56366bdf6c79
Successfully built snownlp
Note: you may need to restart the kernel to use updated packages.


In [4]:
from snownlp import SnowNLP
from snownlp import sentiment

In [5]:
import chardet

In [9]:
# 查看文件编码格式
with open(r'./sentimentanalysis/neg60000.txt', 'rb+') as fp:
    content = fp.read()
    print(chardet.detect(content))

# 查看文件编码格式
with open(r'./sentimentanalysis/pos60000.txt', 'rb+') as fp:
    content = fp.read()
    print(chardet.detect(content))

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}
{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [6]:
# 如果文件格式为GB2312，需修改文件编码格式为utf-8
# 已经修改为utf-8之后，下面的代码就无需再运行
#pos_file = r'./sentimentanalysis/pos60000.txt'
#with open(pos_file, 'rb+') as f:
    # 处理的字符的确是gb2312，但是其中夹杂的部分特殊字符，是gb2312编码中所没有的
    # 所以，此种情况，可以尝试用和当前编码（gb2312）兼容的但所包含字符更多的编码（gb18030）去解码就可以
    #data = f.read().decode('gb18030').encode('utf-8')
    #f.seek(0)
    #f.truncate()
    #f.write(data)

In [10]:
# 使用新数据重新训练模型
if __name__ == "__main__":
    # 重新训练模型
    sentiment.train('./sentimentanalysis/neg60000.txt','./sentimentanalysis/pos60000.txt')
    # 保存好新训练的模型
    sentiment.save('sentiment.marshal')

In [12]:
s=SnowNLP(u'当年长春围城多长时间？')
print(s.words)   #返回分词结果
print(s.sentiments)   #返回该句话的情感得分

['当年', '长春', '围城', '多', '长', '时间', '？']
0.21290258249463978


In [237]:
input_path = './classify/huxijin/all_file/'
output_path = './classify/huxijin/out_result/'
files = os.listdir(input_path)
print(files)

file_names = []
# 去除隐藏文件.DS_Store
for file in files:
    if file.endswith('.csv'):
        file_names.append(file)
    if not file.endswith('.csv'):
        continue
    print(file_names)
    
for file in file_names:
    mydf = pd.read_csv(input_path + file, index_col = None)
    mydf = pd.DataFrame(mydf)
    
    #print(mydf.dtypes)
    
    # mydf['评论内容'] = mydf['评论内容'].apply(str) # 将‘评论内容’列的格式从object转为str
    mydf['评论内容'] = mydf['评论内容'].fillna('无') # fillna 填补空值
    replies = mydf['评论内容'] # 读取每个csv文件的‘评论内容’列
    
    reply_content = [] # 建立空列存储每条评论，方便后续对照
    reply_score = [] # 建立空列存储每条评论对应的sentiment score
    for n in replies:
        a1 = SnowNLP(n)
        a2 = a1.sentiments
        reply_content.append(n)
        reply_score.append(a2)
    
    table = pd.DataFrame({'reply_content': reply_content, 'sentiment': reply_score})
    # 计算‘sentiment score’列的中位数，存放在'sentiment median'列中
    table['sentiment_median'] = table['sentiment'].median()
    # 计算单条微博评论的affective polarization值，存放在‘polarization’列中
    table['polarization'] = table.apply(lambda x: abs(x['sentiment'] - x['sentiment_median']), axis = 1)
    # 计算该条微博下整体的affective polarization值（即为所有评论的polarization值的中位数）
    # 存放在affective polarization列中
    table['affective_polarization'] = table['polarization'].median()
    # 建立新列，存放微博的独立网址ID
    table['WeiboID'] = input_path + file
    
    #print(table.dtypes)
    
    match = table['WeiboID'].str.findall(r'./classify/huxijin/all_file/(.*?).csv') # 提取微博ID（纯数字）
    if len(match) > 0:
        table['weiboID'] = match[0][0]
    table['weiboID'] = 'https://weibo.com/1989660417/' + table['weiboID'] + '?from=page_1035051989660417_profile&wvr=6&mod=weibotime'
    #print(table)
    
    table.to_csv(output_path + file + '.csv')

['LndHXAPjM.csv', 'Lq6j03OQt.csv', 'Lne66viLR.csv', 'Ln02la62c.csv', 'LnFi6jUBQ.csv', 'LrTUFFa4B.csv', 'LoDbcAy7t.csv', 'LorycgK5d.csv', 'LnMA4FpDL.csv', 'LmrOviI8X.csv', 'LqRWDojJi.csv', 'Lrvv0zAtu.csv', 'LqreApS6m.csv', 'Lr1NyqEpv.csv', 'Lqhj9dOv3.csv', 'Lo9iD4Nve.csv', 'LrsU5gaql.csv', 'LrTburBmY.csv', 'LpWoJAeoP.csv', 'Lra4nosoe.csv', 'Lofv6obM5.csv', 'LpnlLg92F.csv', 'LnS5onx5T.csv', 'LpiKb8LQi.csv', 'Lq3WMFJAO.csv', 'LsIq7q2Sc.csv', 'LseQpsB4t.csv', 'LoKeG8ws4.csv', 'Lo0Rh1sVE.csv', 'LpTJQ74ZW.csv', 'Lnn0kCVSR.csv', 'LnmrQ9u1l.csv', 'LspQJh0qo.csv', 'LpLIf1RgW.csv', 'LqTyg5N7V.csv', 'Lswo8crsN.csv', 'LsxvI2vn5.csv', 'Lqmpd78Ll.csv', 'LpvhSE1Wx.csv', 'Ls4J1vPGU.csv', 'LoggWzgrt.csv', 'LmMG9vPLk.csv', 'Lor79B2OO.csv', 'LsnOG7KXI.csv', 'LnYYO55RS.csv', 'LrCLT1UJ2.csv', 'Lo79msohp.csv', 'LsccS1gtp.csv', 'LqoKJoKFz.csv', 'Lr7cS42qZ.csv', 'LnHPtw0yk.csv', 'LsH5DpDin.csv', 'Lo6bOE4Qo.csv', 'LnydsrH39.csv', 'LrVYoidho.csv', 'LsecV1Xvf.csv', 'Lqchjb8Ar.csv', 'Lqpkv4Ykn.csv', 'LsfwraZUi.cs

In [238]:
# Combine all csv files in h1-merge

input_path = './classify/huxijin/out_result/'
files = os.listdir(input_path)
print(files)

file_names = []
# 去除隐藏文件.DS_Store
for file in files:
    if file.endswith('.csv'):
        file_names.append(file)
    if not file.endswith('.csv'):
        continue
    print(file_names)

all_data_frames = []
for file in file_names:
    data_frame = pd.read_csv(input_path + file, index_col = None)
    all_data_frames.append(data_frame)

data_frame_concat = pd.concat(all_data_frames, axis = 0, ignore_index = True)
print(data_frame_concat)
#data_frame_concat.to_csv('test_result.csv')

['LnkJka9IL.csv.csv', 'LnYadsf8w.csv.csv', 'Lor79B2OO.csv.csv', 'Lo79msohp.csv.csv', 'Lq3WMFJAO.csv.csv', 'LqyJom5fX.csv.csv', 'LpiKb8LQi.csv.csv', 'LpvZP4iIv.csv.csv', 'LoggWzgrt.csv.csv', 'LmJ5v47uL.csv.csv', 'LsnOG7KXI.csv.csv', 'Lne66viLR.csv.csv', 'LniLJyo4O.csv.csv', 'Lnn0kCVSR.csv.csv', 'Lo9iD4Nve.csv.csv', 'LrTburBmY.csv.csv', 'Lrvv0zAtu.csv.csv', 'LpTJQ74ZW.csv.csv', 'Lnl1mjIas.csv.csv', 'Lqhj9dOv3.csv.csv', 'LnHPtw0yk.csv.csv', 'LsH5DpDin.csv.csv', 'Lqchjb8Ar.csv.csv', 'LsfwraZUi.csv.csv', 'Lra4nosoe.csv.csv', 'LmMG9vPLk.csv.csv', 'LpZjhq5SU.csv.csv', 'LmrOviI8X.csv.csv', 'LqreApS6m.csv.csv', 'LpvhSE1Wx.csv.csv', 'LseQpsB4t.csv.csv', 'LsFaaiBsz.csv.csv', 'LrL06id9y.csv.csv', 'LqTyg5N7V.csv.csv', 'LoKeG8ws4.csv.csv', 'LrCLT1UJ2.csv.csv', 'LsxvI2vn5.csv.csv', 'LnydsrH39.csv.csv', 'Lp7QndkNJ.csv.csv', 'Lqg7MuXz1.csv.csv', 'Lo6bOE4Qo.csv.csv', 'Lr7cS42qZ.csv.csv', 'LmxRZzKoj.csv.csv', 'Lo0Rh1sVE.csv.csv', 'LokOM8DFP.csv.csv', 'LnMA4FpDL.csv.csv', 'LnoGVrP5g.csv.csv', 'Lr1NyqEpv.c

       Unnamed: 0              reply_content  sentiment  sentiment_median  \
0               0              西方太狂了，我们应该反击！   0.090748          0.273127   
1               1                  老胡说得对，干他！   0.325343          0.273127   
2               2                     建议的真好！   0.498751          0.273127   
3               3  老胡 先把宝山区人民的肚子填饱再来讨论台湾问题行不   0.161228          0.273127   
4               4             不怕，我们有红线最终解释权；   0.442965          0.273127   
...           ...                        ...        ...               ...   
69065         559            原本以为就是个天灾……命苦啊！   0.013296          0.293826   
69066         560                     建议火速封城   0.636125          0.293826   
69067         561                    五里桥怎么样啦   0.500000          0.293826   
69068         562                        封城吧   0.452637          0.293826   
69069         563                      听天由命了   0.333333          0.293826   

       polarization  affective_polarization  \
0          0.182379         

In [240]:
df_classified = pd.read_csv('./Weibo/huxijin.csv', index_col = None)
# 修改列名
df_classified = df_classified.rename(columns = {'博文独立网址': 'weiboID'})

# 对其weiboID，使用inner合并的方式，将每条微博下评论的情感分数对应到相应的微博下
df_merge = pd.merge(df_classified, data_frame_concat, on = 'weiboID', how = 'inner')

# 使用drop_duplicates中的subset参数删除'博文'列重复的数据
df_merge = df_merge.drop_duplicates(subset = '博文')
print(df_merge)

df_merge.to_csv('huxijin_affectivepolar_result.csv')

        博主           简介  关注数      粉丝数    微博数 发博博主  \
0      胡锡进  《环球时报》特约评论员  296  2439.2万  10481  胡锡进   
377    胡锡进  《环球时报》特约评论员  296  2439.2万  10481  胡锡进   
748    胡锡进  《环球时报》特约评论员  296  2439.2万  10481  胡锡进   
1411   胡锡进  《环球时报》特约评论员  296  2439.2万  10481  胡锡进   
2037   胡锡进  《环球时报》特约评论员  296  2439.2万  10481  胡锡进   
...    ...          ...  ...      ...    ...  ...   
67010  胡锡进  《环球时报》特约评论员  296  2439.2万  10481  胡锡进   
68784  胡锡进  《环球时报》特约评论员  296  2439.2万  10481  胡锡进   
68819  胡锡进  《环球时报》特约评论员  296  2439.2万  10481  胡锡进   
70667  胡锡进  《环球时报》特约评论员  296  2439.2万  10481  胡锡进   
71155  胡锡进  《环球时报》特约评论员  296  2439.2万  10481  胡锡进   

                                                 发博博主链接  \
0      https://weibo.com/huxijin?refer_flag=1005055013_   
377    https://weibo.com/huxijin?refer_flag=1005055013_   
748    https://weibo.com/huxijin?refer_flag=1005055013_   
1411   https://weibo.com/huxijin?refer_flag=1005055013_   
2037   https://weibo.com/huxijin?refer_flag=1005055013_   
...      

### 2.2 Classifying Ukraine-Russia-related Weibo Posts

In [193]:
import csv
import re
import os
import jieba.posseg as pseg
import pandas as pd
import numpy as np
import glob
import operator # for sorting dictionaries

In [194]:
# Define a function to preprocess the texts
#
# Prepare the stopwords
stopwords = list()
for file in os.listdir("./stop_common/"):
    if ".DS_" not in file:
        with open("./stop_common/" + file, "r", encoding="utf-8") as f:
            for line in f.readlines():
                stopwords.append(line.strip())
print(len(stopwords))

stopwords = list(set(stopwords))
print(len(stopwords))

# Load the self-defined dictionary
selfdict = list()
with open("./dict-Ukraine-Russia.txt", "r", encoding = 'utf-8') as f:
    for line in f.readlines():
        selfdict.append(line.strip())
print(len(selfdict))

def tokenize(text):
    kept_words = list()
    for word, flag in pseg.cut(text):
        # Filter stopwords and words whose freqs < 1
        if (word not in stopwords) and \
        (len(word) > 1):
            kept_words.append(word)
    return " ".join(kept_words)

7938
2433
493


In [222]:
# LOAD AND CLEAN DATA

# Load in the input file and process each row at a time.
# The file has these columns that are meaningful in the research:
# 7: "博文"
# 8: "发布时间"
# 9: "博文独立网址"
# 11: "转发数"
# 12: "评论数"
# 13: "点赞数"
#
# Create a data structure for each Weibo post:
# orig: The original, unpreprocessed string of characters
# pubdate: The publication date of the Weibo post
# link: The web link of the Weibo post
# clean: The preprocessed string of characters
# trans: The transmitting freqs of the Weibo post
# comm: The comments that each Weibo post received
# likes: The number of likes that each Weibo post received
#
# Intialize an empty list to hold all of our posts
posts = []

with open('./Weibo/caixin.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter = ',')
    next(reader)
    for row in reader:
        
        post = dict()
        post['orig'] = row[7]
        post['pubdate'] = row[8]
        post['link'] = row[9]
        post['trans'] = row[11]
        post['comm'] = row[12]
        post['likes'] = row[13]
        
        post['clean'] = post['orig']
        
        # Preprocessing posts
        post['clean'] = tokenize(post['clean'])
        # Remove the hashtag symbol
        post['clean'] = post['clean'].replace(r'#', '')
        # Remove NaN of comments
        post['comm'] = post['comm'].replace(r'评论', '0')
        
        posts.append(post)

In [223]:
# Create a data structure to hold the lexicon.
# We will use a Python diction. The key of the dictionary will be the word.
# and the value will be the word's score.
lexicon = dict()

# Read in the lexicon.
with open('./Ukraine-Russia-War-lexicon.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter = ',')
    for row in reader:
        lexicon[row[0]] = int(row[1])

In [224]:
# Use the lexicon to score posts
for post in posts:
    score = 0
    for word in post['clean'].split():
        if word in lexicon:
            score = score + lexicon[word]
            
    post['score'] = score
    if (score > 0):
        post['classified'] = '1'
    elif (score < 0):
        post['classified'] = '0'
    else:
        post['classified'] = '0'

In [225]:
# Print out summary stats
total = float(len(posts))
num_related = sum([1 for t in posts if t['classified'] == '1'])
num_unrelated_unknown = sum([1 for t in posts if t['classified'] == '0'])
print(num_related, 100.0 * (num_related/total))
print(num_unrelated_unknown, 100.0 * (num_unrelated_unknown/total))

84 4.786324786324787
1671 95.2136752136752


In [226]:
posts

[{'orig': '',
  'pubdate': '',
  'link': '',
  'trans': '',
  'comm': '',
  'likes': '',
  'clean': '',
  'score': 0,
  'classified': '0'},
 {'orig': '【#20国报告348例儿童不明原因肝炎# 病因仍待确认】全球报告的儿童不明原因肝炎病例数继续增多，升至348例，死亡8例。至5月10日，世界卫生组织（WHO）一共接到了20多国报告的共348例儿童急性重症不明原因肝炎病例，另有70例疑似病例待确认，8个死亡病例包括美国报告的5个和印度尼西亚报告的3个。20个 \u200b\u200b\u200b\u200b...展开全文c',
  'pubdate': '2022-05-12 22:47',
  'link': 'https://weibo.com/1663937380/LsIi67oeG?from=page_1002061663937380_profile&wvr=6&mod=weibotime',
  'trans': '9',
  'comm': '20',
  'likes': '86',
  'clean': '20 报告 348 儿童 不明 原因 肝炎 病因 确认 全球 报告 儿童 不明 原因 肝炎 病例 增多 升至 348 死亡 10 世界卫生组织 WHO 一共 接到 20 多国 报告 348 儿童 急性 重症 不明 原因 肝炎 病例 70 疑似病例 确认 死亡 病例 包括 美国 报告 印度尼西亚 报告 20',
  'score': -5,
  'classified': '0'},
 {'orig': '【#银河系中心首张黑洞照片公布# 质量约为太阳430万倍】银河系中心黑洞的“庐山真面目”终浮现。北京时间5月12日21时许，中国科学院上海天文台和事件视界望远镜（EHT）组织共同发布了银河系中心黑洞人马座A*的首张照片。这一黑洞的质量约为太阳质量的430万倍，距离地球约2.7万光年。虽然人们无法看到 \u200b\u200b\u200b\u200b...展开全文c',
  'pubdate': '2022-05-12 22:29',
  'link': 'https://weibo.com/166393

In [227]:
column_name = ['orig', 'pubdate', 'link', 'trans', 'comm', 'likes', 'clean', 'score', 'classified']
posts_csv = pd.DataFrame(columns = column_name, data = posts)
# Delete the row which index = 0
posts_csv = posts_csv.drop(0, axis = 0)
# Add a new column for its account name
posts_csv['account'] = '财新网'

In [228]:
# Split the pubdate column into two columns
posts_csv[['date', 'time']] = posts_csv['pubdate'].str.split(' ', expand=True)
# Delete the time column
posts_csv = posts_csv.drop(labels = 'time', axis = 1)

# Translate the datatype of column 'classified', 'comm' from str to int
posts_csv['classified'] = posts_csv['classified'].astype('int')

# Translate the datatype of column 'classified', 'comm' from object to int
posts_csv['comm'] = pd.to_numeric(posts_csv['comm'])

# Translate the datatype of column 'date', 'comm' from object to datetime
posts_csv['date'] = pd.to_datetime(posts_csv['date'])

In [229]:
posts_csv

,orig,pubdate,link,trans,comm,likes,clean,score,classified,account,date
1,【#20国报告348例儿童不明原因肝炎# 病因仍待确认】全球报告的儿童不明原因肝炎病例数继续...,2022-05-12 22:47,https://weibo.com/1663937380/LsIi67oeG?from=pa...,9,20.0,86,20 报告 348 儿童 不明 原因 肝炎 病因 确认 全球 报告 儿童 不明 原因 肝炎 ...,-5,0,财新网,2022-05-12
2,【#银河系中心首张黑洞照片公布# 质量约为太阳430万倍】银河系中心黑洞的“庐山真面目”终浮...,2022-05-12 22:29,https://weibo.com/1663937380/LsIaV6JRE?from=pa...,24,23.0,78,银河系 中心 首张 黑洞 照片 公布 质量 太阳 430 万倍 银河系 中心 黑洞 庐山真面...,-2,0,财新网,2022-05-12
3,#财新图片#【视线｜“封城、静默”被辟谣前，北京市民采购忙】5月12日，朝阳区青年路附近一超...,2022-05-12 22:03,https://weibo.com/1663937380/LsI0ypZ3w?from=pa...,20,27.0,156,财新 图片 视线 封城 静默 辟谣 北京市民 采购 朝阳区 青年 超市 结账 队伍 商品 市...,-8,0,财新网,2022-05-12
4,【财务总监拒签财报失联 福成股份回应最新进展】5月11日晚间，福成股份（ 600965.SH...,2022-05-12 21:40,https://weibo.com/1663937380/LsHRijNQ5?from=pa...,7,5.0,21,财务总监 拒签 报失 福成 股份 回应 最新进展 晚间 福成 股份 600965. SH 发...,-3,0,财新网,2022-05-12
5,【疫情冲击需求叠加产品涨价 #宁德时代4月装车量环比腰斩#】4月，宁德时代（ 300750....,2022-05-12 21:20,https://weibo.com/1663937380/LsHIR38u7?from=pa...,6,3.0,13,疫情 冲击 需求 叠加 产品 涨价 宁德 时代 装车 环比 腰斩 宁德 时代 300750....,-4,0,财新网,2022-05-12
...,...,...,...,...,...,...,...,...,...,...,...
1750,【3月#财新中国制造业PMI#降至48.1 为2020年3月以来最低】3月新冠疫情在全国多地...,2022-04-01 10:05,https://weibo.com/1663937380/LmoMN2J6y?from=pa...,24,39.0,125,财新 中国 制造业 PMI 降至 48.1 2020 最低 新冠 疫情 全国 蔓延 防疫 措...,-6,0,财新网,2022-04-01
1751,今日A股敏感舆情榜，华夏幸福公告最新债务重组实施进展，排名第一；福达合金因拟进行重大资产置换...,2022-04-01 09:39,https://weibo.com/1663937380/LmoCdxZyn?from=pa...,转发,0.0,1,今日 A股 敏感 舆情 华夏 幸福 公告 最新 债务 重组 实施 进展 排名 福达 合金 资...,-4,0,财新网,2022-04-01
1752,【上海封“半城”下，一个哮喘病人的急诊经过】浦东“封城”第二天，3月29日晚上7点40分，居...,2022-04-01 09:12,https://weibo.com/1663937380/Lmor3AYc1?from=pa...,18,60.0,339,上海 半城 哮喘病 急诊 浦东 封城 第二天 29 晚上 40 居住 浦东 康桥 街道 郑明...,-1,0,财新网,2022-04-01
1753,【#中国石油2021年净利922亿元# 创七年新高】2021年，油气产品价格大涨，中国石油天...,2022-04-01 08:51,https://weibo.com/1663937380/LmoiFx1zr?from=pa...,19,42.0,227,中国 石油 2021 净利 922 亿元 七年 新高 2021 油气 产品价格 中国 石油 ...,-2,0,财新网,2022-04-01


In [230]:
posts_csv.dtypes

orig                  object
pubdate               object
link                  object
trans                 object
comm                 float64
likes                 object
clean                 object
score                  int64
classified             int64
account               object
date          datetime64[ns]
dtype: object

In [122]:
posts_csv_grouped = posts_csv.groupby('date')
posts_grouped_mean = posts_csv_grouped.mean()

# Insert a new column of account name
## 0: the inserting place in the columns
## 'account': the name of the inserting column
## '人民日报': the content of the inserting column
posts_grouped_mean.insert(0, 'account', '新华网')

# Insert a new publish-date column
date_range = pd.date_range(start = '2022-04-01', end = '2022-05-12')
posts_grouped_mean.insert(1, 'publish_date', date_range)

# Delete the index
posts_grouped_mean = posts_grouped_mean.reset_index(drop = True)

print(posts_grouped_mean.dtypes)
posts_grouped_mean

account                 object
publish_date    datetime64[ns]
comm                   float64
score                  float64
classified             float64
dtype: object


,account,publish_date,comm,score,classified
0,新华网,2022-04-01,52.590909,-1.022727,0.136364
1,新华网,2022-04-02,136.877551,-2.061224,0.122449
2,新华网,2022-04-03,105.295455,-3.045455,0.090909
3,新华网,2022-04-04,410.568182,-4.181818,0.022727
4,新华网,2022-04-05,101.936170,-3.106383,0.021277
5,新华网,2022-04-06,36.745098,-3.392157,0.098039
6,新华网,2022-04-07,289.634615,-2.692308,0.153846
7,新华网,2022-04-08,135.855556,-3.577778,0.066667
8,新华网,2022-04-09,91.166667,-4.761905,0.071429
9,新华网,2022-04-10,91.375000,-2.850000,0.025000


In [123]:
# Mean value imputation '胡锡进'(均值插补法)
# 只在“胡锡进”的微博主页数据里使用，因为胡锡进的账号在4.15和4.18有缺失值
### 2022-04-15
#posts_grouped_mean.loc[14, 'comm'] = [4419.710417]
#posts_grouped_mean.loc[14, 'score'] = [-2.654167]
#posts_grouped_mean.loc[14, 'classified'] = [0.156250]

### 2022-04-18
#posts_grouped_mean.loc[17, 'comm'] = [4419.710417]
#posts_grouped_mean.loc[17, 'score'] = [-2.654167]
#posts_grouped_mean.loc[17, 'classified'] = [0.156250]

In [126]:
# Select the placebo test group and control group between 2022-04-01 and 2022-04-28
data_control = posts_grouped_mean[(posts_grouped_mean['publish_date'] >= pd.to_datetime('2022-04-01')) & (posts_grouped_mean['publish_date'] <= pd.to_datetime('2022-04-28'))]

# Insert a new column of treatment status (placebo data set & control group)
data_control.insert(2, 'treatment_status', 0)

# Select the treatment group
data_treated = posts_grouped_mean[(posts_grouped_mean['publish_date'] >= pd.to_datetime('2022-04-29'))]

# Insert a new column of treatment status
data_treated.insert(2, 'treatment_status', 1)

xinhua_merge = data_control.append(data_treated)
xinhua_merge

,account,publish_date,treatment_status,comm,score,classified
0,新华网,2022-04-01,0,52.590909,-1.022727,0.136364
1,新华网,2022-04-02,0,136.877551,-2.061224,0.122449
2,新华网,2022-04-03,0,105.295455,-3.045455,0.090909
3,新华网,2022-04-04,0,410.568182,-4.181818,0.022727
4,新华网,2022-04-05,0,101.936170,-3.106383,0.021277
5,新华网,2022-04-06,0,36.745098,-3.392157,0.098039
6,新华网,2022-04-07,0,289.634615,-2.692308,0.153846
7,新华网,2022-04-08,0,135.855556,-3.577778,0.066667
8,新华网,2022-04-09,0,91.166667,-4.761905,0.071429
9,新华网,2022-04-10,0,91.375000,-2.850000,0.025000


In [127]:
xinhua_merge.to_csv('./h1-merge/xinhua_merge.csv', encoding = 'utf-8', index = False)

## 3.0 Combine engagement data with affective polarization degree data

In [329]:
affective = pd.read_csv('./h1-merge/affectivepolar/xinhua_affectivepolar_result.csv')
affective_df = pd.DataFrame(affective)

# Split the pubdate column into two columns
affective_df[['date', 'time']] = affective_df['发布时间'].str.split(' ', expand=True)
# Delete the time column
affective_df = affective_df.drop(labels = 'time', axis = 1)
affective_df

# 按照发布日期给affective polarization分组，并求其当天的均值
affective_df_grouped = affective_df.groupby('date')
affective_df_grouped_mean = affective_df_grouped.mean()

# Delete the index
# affective_df_grouped_mean = affective_df_grouped_mean.reset_index(drop = True)

In [284]:
#------------非胡锡进账号数据不需要运行以下内容------------#
#------------非胡锡进账号数据不需要运行以下内容------------#
#------------非胡锡进账号数据不需要运行以下内容------------#

# 由于胡锡进的数据存在缺失值，所以需要在04-15，04-17和04-22三天进行插补
# 其余媒体账号的数据不必执行如下步骤
## 此处采用的是均值插补法，先算出其余26天affective polarization的均值（0.167824226），然后插入到dataframe中

def insert(df, i, df_add):
    # 指定第i行插入一行数据
    df1 = df.iloc[:i, :]
    df2 = df.iloc[i:, :]
    df_new = pd.concat([df1, df_add, df2], ignore_index=True)
    return df_new

# 在第15行插入一条新的数据
df_add_15 = pd.DataFrame({'Unnamed: 0': ['71155.000000'], '关注数': ['296.0'], '微博数': ['10481.0'], '转发数': ['233.000000'], '评论数': ['1203.0'], '点赞数': ['5544.0'], '转发_转发数': ['NaN'],
                          '转发_评论数': ['NaN'], '转发_点赞数': ['NaN'], '是否为点赞微博': ['NaN'], '是否为快转微博': ['NaN'], 'Unnamed: 0.1': ['0.0'], 'sentiment': ['0.346231'], 'sentiment_median': ['0.337881'],
                         'polarization': ['0.00835'], 'affective_polarization': ['0.167824']})
affective_df_grouped_mean = insert(affective_df_grouped_mean, 14, df_add_15)

# 在第17行插入一条新的数据
df_add_17 = pd.DataFrame({'Unnamed: 0': ['71155.000000'], '关注数': ['296.0'], '微博数': ['10481.0'], '转发数': ['233.000000'], '评论数': ['1203.0'], '点赞数': ['5544.0'], '转发_转发数': ['NaN'],
                          '转发_评论数': ['NaN'], '转发_点赞数': ['NaN'], '是否为点赞微博': ['NaN'], '是否为快转微博': ['NaN'], 'Unnamed: 0.1': ['0.0'], 'sentiment': ['0.346231'], 'sentiment_median': ['0.337881'],
                         'polarization': ['0.00835'], 'affective_polarization': ['0.167824']})
affective_df_grouped_mean = insert(affective_df_grouped_mean, 16, df_add_17)

# 在第22行插入一条新的数据
df_add_22 = pd.DataFrame({'Unnamed: 0': ['71155.000000'], '关注数': ['296.0'], '微博数': ['10481.0'], '转发数': ['233.000000'], '评论数': ['1203.0'], '点赞数': ['5544.0'], '转发_转发数': ['NaN'],
                          '转发_评论数': ['NaN'], '转发_点赞数': ['NaN'], '是否为点赞微博': ['NaN'], '是否为快转微博': ['NaN'], 'Unnamed: 0.1': ['0.0'], 'sentiment': ['0.346231'], 'sentiment_median': ['0.337881'],
                         'polarization': ['0.00835'], 'affective_polarization': ['0.167824']})
affective_df_grouped_mean = insert(affective_df_grouped_mean, 21, df_add_22)

In [330]:
# Insert a new publish-date column
date_range = pd.date_range(start = '2022-04-01', end = '2022-05-12')
affective_df_grouped_mean.insert(1, 'publish_date', date_range)

# Delete the index
affective_df_grouped_mean = affective_df_grouped_mean.reset_index(drop = True)
# Delete other useless columns
affective_df_grouped_mean.drop(columns = ['Unnamed: 0','关注数', '粉丝数', '点赞数', '转发_评论数', '转发_点赞数', '转发_转发数', '微博数', '是否为点赞微博', 'Unnamed: 0.1', 'sentiment', 'sentiment_median', 'polarization'],inplace = True)

affective_df_grouped_mean

,publish_date,affective_polarization
0,2022-04-01,0.131885
1,2022-04-02,0.138406
2,2022-04-03,0.127285
3,2022-04-04,0.140181
4,2022-04-05,0.131585
5,2022-04-06,0.138406
6,2022-04-07,0.108976
7,2022-04-08,0.131348
8,2022-04-09,0.148050
9,2022-04-10,0.161971


In [331]:
# 提取分组后的dataframe中的‘affective_polarization’列，使其成为一个新的dataframe
#affective_polarization_df = affective_df_grouped_mean[['affective_polarization']]

# 读取之前已经处理好的胡锡进engagement数据，将两个dataframe拼接起来
engagement = pd.read_csv('./h1-merge/engagement/xinhua_merge.csv')
engagement_df = pd.DataFrame(engagement)
engagement_df

# 使用concat函数按列合并两个表格
final_merge_df = pd.concat([engagement_df, affective_df_grouped_mean], axis = 1)
final_merge_df
final_merge_df.to_csv('xinhua-merge.csv')

## 4.0 找出在第二步未能成功计算出情感值的文件

In [104]:
pip install xlwt

Note: you may need to restart the kernel to use updated packages.


In [147]:
import shutil
import xlwt

In [164]:
## 首先，先把已经计算出情感值的文件名写入到一个excel中
import xlrd
from xlwt import * # 操作excel模块
import sys

file_path = sys.path[0]+'/chinadaily-allfile-namelist.xls'#sys.path[0]为要获取当前路径，filenamelist为要写入的文件
source_path_1 = './classify/chinadaily/out_result_1/' # 已经完成情感值计算的文件所在地址
source_path_2 = './classify/chinadaily/all_file/' # 所有需要计算情感值的文件所在地址

f = xlwt.Workbook(encoding = 'utf-8', style_compression = 0) # 新建一个excel
sheet = f.add_sheet('sheet1') # 新建一个sheet

pathDir = os.listdir(source_path_2) # 

i = 0
for s in pathDir:
    sheet.write(i, 0, s) # 参数i,0,s分别代表行，列，写入值
    i = i + 1

print(file_path)
print(i)
f.save(file_path)

/Users/liyuhan/IPadress-polarization/chinadaily-allfile-namelist.xls
1884


In [168]:
outresult = pd.read_excel('./chinadaily-outresult-namelist.xls')
outresult_df = pd.DataFrame(outresult)

allfile = pd.read_excel('./chinadaily-allfile-namelist.xls')
allfile_df = pd.DataFrame(allfile)

In [169]:
def same_element(allfile_df, outresult_df):
    col_list = allfile_df.columns.tolist()
    for i in col_list:
        set1 = set(allfile_df[i])
        set2 = set(outresult_df[i])
        print(i,'相同的元素：', (set1 & set2),  i,'相同元素的个数：', len(set1 & set2))
        print(i,'不同的元素：', (set1 ^ set2),  i, '不相同的元素个数：', len(set1 ^ set2))
        print(i,'allfile有,outresult没有的元素：', ((set1|set2)-set2),  i,'allfile有,outresult没有的元素元素个数：', len(((set1|set2)-set2)))
        print(i,'allfile没有,outresult有的元素：', ((set1|set2)-set1),  i, 'allfile没有,outresult有的元素个数：', len(((set1|set2)-set1)))

same_element(allfile_df, outresult_df)

namelist 相同的元素： {'LnqbNsBHv.csv', 'LqR5sil71.csv', 'Lqrgb8MbT.csv', 'LmSVBhZtx.csv', 'LrJ5Eswxv.csv', 'LnaRLuNhV.csv', 'Lrb7xnCOQ.csv', 'Lo3E6bzJU.csv', 'LopFpvgPc.csv', 'LnlCF9RLo.csv', 'LmHZKDMJF.csv', 'Lnx8ewIQJ.csv', 'LrgesC3Om.csv', 'LsHZFutvO.csv', 'Ln4Ieg5U8.csv', 'Lp0YDcMOj.csv', 'LodjAkITd.csv', 'Lq2sOhzF8.csv', 'LoJJV02s2.csv', 'LmKKfzLO5.csv', 'LprCQyiXi.csv', 'LpOHG4lin.csv', 'Lncj52hmB.csv', 'LmVSfFQud.csv', 'LqbKco3dI.csv', 'LondzFPCz.csv', 'LptodtdP1.csv', 'LpFFhqXOh.csv', 'LnGWGbrBG.csv', 'LrHwJzhrm.csv', 'LmW4r27CB.csv', 'LoPNPzWDS.csv', 'LoZVFEhco.csv', 'Lp8EBapz6.csv', 'Lnvw2mQ3Q.csv', 'LpjLykmx9.csv', 'LqoMI1zB1.csv', 'LpMtMg0DM.csv', 'Ls2l4x83T.csv', 'LpNGx1tW8.csv', 'LrDpn8JXd.csv', 'LqPH03Nhx.csv', 'LnaXRao8B.csv', 'LpVIybi1t.csv', 'LqmjJfTXL.csv', 'LrKhfsUkV.csv', 'LsbW7EyfO.csv', 'LnrJtqAzo.csv', 'LpuCCj5i9.csv', 'Lp4gHBxeE.csv', 'Ls0YmejKt.csv', 'Lnv64eIXK.csv', 'Lo8LN7gkg.csv', 'LsmLUrPAt.csv', 'LoUGXwquf.csv', 'LslRTcUiw.csv', 'Lnj7dklxK.csv', 'LsxbhBVXd.csv

In [170]:
col_list = allfile_df.columns.tolist()
for i in col_list:
    set1 = set(allfile_df[i])
    set2 = set(outresult_df[i])

set3 = (set1|set2)-set2
set3_list = list(set3) # 将set转换为list
# 将list转换为dataframe
set_df = pd.DataFrame(set3_list, columns = ['notyet_namelist'])
set_df.to_excel('chinadaily-notyet-namelist.xls', index = None)

/Users/liyuhan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  # Remove the CWD from sys.path while we load stuff.


In [171]:
import shutil
import xlrd

file_source_path = './classify/chinadaily/all_file' # 源文件夹路径
file_target_path = './classify/chinadaily/notyet_file' # 目标文件夹路径

notyet = xlrd.open_workbook_xls('chinadaily-notyet-namelist.xls')
notyet_rsheet = notyet.sheet_by_index(0) # 获取第一个sheet页

for row in notyet_rsheet.get_rows():
    notyet_name_col = row[0]
    notyet_name = notyet_name_col.value # 获取待处理的文件的标题
    if notyet_name != 'notyet_namelist': # 排除第一行
        notyet_name = f'/{notyet_name}'
        source_file = file_source_path + notyet_name
        target_file = file_target_path + notyet_name
        
        if os.path.exists(source_file):
            shutil.copy(source_file, target_file)
        else:
            print("文件" + source_file + "不存在")

In [334]:
# Combine all csv files in h1-merge
csv_list = glob.glob('./h1-merge/final_merge/*.csv')
print(len(csv_list))

6


In [335]:
for i in csv_list: #循环读取同文件夹下的csv文件
    fr = open(i,'rb').read()
    with open('affective_engagement_merge_new.csv','ab') as f: #将结果保存为result.csv
        f.write(fr)
print(u'合并完毕！')

合并完毕！
